In [10]:
import torch
import pandas as pd

In [2]:
experts: dict[str, torch.Tensor] = torch.load(
    "/home/joe/Mixtral-8x7B-Instruct-v0.1-Official/experts.pt",
    map_location=torch.device("cpu"),
    mmap=True,
)

/tmp/ipykernel_4183777/1870126095.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  experts: dict[str, torch.Tensor] = torch.load(


In [56]:
top1_avgs = torch.tensor([0.74609375, 0.7265625, 0.70703125, 0.6875, 0.69140625, 0.69140625, 0.67578125, 0.6875, 0.68359375, 0.67578125, 0.67578125, 0.7265625, 0.71875, 0.65625, 0.6875, 0.7109375, 0.6953125, 0.71875, 0.7109375, 0.7265625, 0.75, 0.75, 0.75, 0.765625, 0.765625, 0.7734375, 0.76953125, 0.78125, 0.7734375, 0.77734375, 0.7421875, 0.7109375], dtype=torch.bfloat16)
top2_avgs = torch.tensor([0.25390625, 0.2734375, 0.29296875, 0.3125, 0.306640625, 0.30859375, 0.322265625, 0.314453125, 0.314453125, 0.32421875, 0.32421875, 0.2734375, 0.283203125, 0.34375, 0.314453125, 0.2890625, 0.302734375, 0.28125, 0.291015625, 0.2734375, 0.25, 0.25, 0.248046875, 0.2333984375, 0.232421875, 0.224609375, 0.23046875, 0.21875, 0.2275390625, 0.2236328125, 0.259765625, 0.291015625], dtype=torch.bfloat16)

# top1_model_avg = 0.72216796875
# top2_model_avg = 0.277801513671875

In [57]:
experts["0.0"].shape

torch.Size([3, 14336, 4096])

In [62]:
zeros = torch.zeros((3, 14336, 4096), dtype=torch.bfloat16)
atol = torch.tensor(1e-3, dtype=torch.bfloat16)
data = {"li": [], "ei": [], "n_top1_near_zeros": [], "n_top2_near_zeros": []}
for li in range(32):
    for ei in range(8):
        data["li"].append(li)
        data["ei"].append(ei)
        data["n_top1_near_zeros"].append(
            torch.sum(torch.abs(experts[f"{li}.{ei}"] * top1_avgs[0] - zeros) <= atol).item()
        )
        data["n_top2_near_zeros"].append(
            torch.sum(torch.abs(experts[f"{li}.{ei}"] * top2_avgs[0] - zeros) <= atol).item()
        )
df = pd.DataFrame(data)

In [63]:
df[df["li"] == 31]

,li,ei,n_top1_near_zeros,n_top2_near_zeros
248,31,0,16404463,47239825
249,31,1,16668262,47992534
250,31,2,16618805,47832778
251,31,3,16955733,48762326
252,31,4,16215568,46720979
253,31,5,16617187,47857686
254,31,6,17078780,49127317
255,31,7,16300763,46969800


In [65]:
16404463/(3*14336*4096)

0.09312211331867036

In [67]:
(0.91+0.75)/2

0.8300000000000001